In [ ]:
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.pyplot import figure
from matplotlib.ticker import ScalarFormatter
import glob
import os
import metpy
import metpy.calc as mpcalc
from metpy.units import units
import numpy as np
from datetime import datetime
import xarray as xr
from pint import UnitRegistry
ureg = UnitRegistry()
import seaborn as sns
import scipy as sp
from scipy import stats
from scipy.stats import f_oneway
import statsmodels.api as sm


In [ ]:
# Function to extract epoch from the first few rows of the CSV file
def extract_epoch_from_header(file):
    # Read the first few lines to find the epoch (assuming it's in a comment or a header)
    with open(file, 'r') as f:
        skiprow = f.readline()
        skiprow = skiprow.strip().split()[0]
        for line in f:
            if "startdate:" in line.lower():  # Assuming the word 'epoch' is present in the line
                # Extract the epoch date (this assumes the date is the second item in the line)
                epoch = line.strip().split()[1]
                return int(skiprow)-1, epoch[0:4]+"-"+epoch[4:6]+"-"+epoch[6:8]+" "+epoch[8:10]+":"+epoch[10:12]+":"+epoch[12:14]
    return None  # Return None if no epoch is found

# Function to load and adjust time for each CSV
def load_and_adjust_time(file):
    # Try to extract the epoch from the header or a specific column
    nskiprows,epoch = extract_epoch_from_header(file)
   
    if epoch is None:
        raise ValueError(f"Could not find epoch for file: {file}")
   
    # Load the actual data (skipping header if necessary)
    df = pd.read_csv(file, skiprows=nskiprows,sep="\\s+")  # Adjust skiprows based on where the data starts
   
    # Convert 'time' column to a datetime, assuming it's in days
    # Adjust time column to start from the epoch time found in the file
    df['starttime_dt'] = pd.to_timedelta(df['starttime'], unit='D') + pd.Timestamp(epoch)
   
    # Return the adjusted DataFrame
    return df

In [ ]:
#set a working directory till NMVOC folder
working_dir = "C:\\Year1\\"
# Find all CSV files (change the path if nedeed)
prop_files = glob.glob(working_dir+"NMVOC\\Data\\data_EBAS_propane\\*.nas") 
print(prop_files)

# this function reads apropadjust times for all CSV files
dfs = []
for file in prop_files:
    df = load_and_adjust_time(file)
    dfs.append(df)

# concatenates all DataFrames (based on time)
prop_df = pd.concat(dfs).sort_values('starttime_dt').reset_index(drop=True)

# Show combined DataFrame
print(prop_df)

In [ ]:
df=prop_df
#remove lines depending on flags
df = df[df.flag != 0.999]
df = df[df.flag != 0.899]
df = df[df.flag != 0.456]
df = df[df.flag != 0.260]
df = df[df.flag != 0.259]
df = df[df.flag != 0.256]
df = df[df.C3H8 != 99999.99]

df.index = df["starttime_dt"]
#remove useless columns
df = df.drop(columns=["flag"])
df = df.drop(columns=["starttime"])
df = df.drop(columns=["endtime"])
df = df.drop(columns=["C3H8.1"])
df = df.drop(columns=["C3H8.2"])
df

In [ ]:
# helper columns with  hour, month, year
df['hour'] = df['starttime_dt'].dt.hour
df['month'] = df['starttime_dt'].dt.month
df['year'] = df['starttime_dt'].dt.year
df['date'] = df['starttime_dt'].dt.date
df['day'] = df['starttime_dt'].dt.dayofyear

In [ ]:
#loop for rounding to the hours
df["starttime_dt"] = [i.round("H") for i in df["starttime_dt"]]
#loop for rounding to the hours
df["datetime"]=df["starttime_dt"]
df = df.drop(columns=["starttime_dt"])
prop_df=df

In [ ]:
# Convert date column to datetime
df['date'] = pd.to_datetime(df['date'])



In [ ]:
# Path to the text file
file_path1 =  r"C:\\YEAR1\\NMVOC\\CO_CH4\\co_cmn_surface-insitu_74_2008_2017_event.txt"
file_path2 =  r"C:\\YEAR1\\NMVOC\\CO_CH4\\ICOS_ATC_L2_L2-2024.1_CMN_8.0_CTS.CO"

In [ ]:
#CO dataset between 2018 and 2024
df_24 = pd.read_csv(file_path2, sep=";",skiprows=44)
df_24 ['datetime'] = pd.to_datetime(df_24[['Year', 'Month', 'Day', 'Hour']])
df_24 = df_24[df_24['Flag'] != "N"]  # Filter rows where 'QCflag:comment' equals 3
df_24 = df_24[df_24['Flag']!= "K"] 
df_24 = df_24[df_24['co']!= -9] 
df_24 = df_24.drop(columns=["Flag"]) 
df_24 = df_24.drop(columns=["Site"]) 
df_24 = df_24.drop(columns=["SamplingHeight"]) 
df_24 = df_24.drop(columns=["DecimalDate"]) 
df_24 = df_24.drop(columns=["NbPoints"]) 
df_24 = df_24.drop(columns=["InstrumentId"])
df_24 = df_24.drop(columns=["QualityId"])                            
df_24 = df_24.drop(columns=["LTR"])     
df_24 = df_24.drop(columns=["CMR"])                              
df_24 = df_24.drop(columns=["STTB"])    
df_24 = df_24.drop(columns=["QcBias"])  
df_24 = df_24.drop(columns=["QcBiasUncertainty"])                              
df_24 = df_24.drop(columns=["co-WithoutSpikes"]) 
df_24 = df_24.drop(columns=["Stdev-WithoutSpikes"]) 
df_24 = df_24.drop(columns=["NbPoints-WithoutSpikes"]) 
df_24 = df_24.drop(columns=["Stdev"]) 
df_24 = df_24.drop(columns=["Minute"]) 
df_24.rename(columns={"Year": "year", "Month": "month","Day":"day","Hour":"hour"},inplace=True)
df_24


In [ ]:
#CO dataset till 2017
df_17 = pd.read_csv(file_path1,sep=" ",skiprows=174)
df_17['datetime'] = pd.to_datetime(df_17[['year', 'month', 'day', 'hour']])
df_17 = df_17[df_17['QCflag'] != 3]  # Filter rows where 'QCflag:comment' equals 3
df_17 = df_17[df_17['QCflag']!= -999.999] 
df_17 = df_17[df_17['QCflag']!= -9] 
df_17 = df_17[df_17['year']!= 2008] 
df_17 = df_17[df_17['year']!= 2009] 
df_17 = df_17[df_17['year']!= 2010] 
df_17 = df_17.drop(columns=["nvalue"]) 
df_17 = df_17.drop(columns=["value_unc"]) 
df_17 = df_17.drop(columns=["second"])
df_17 = df_17.drop(columns=["year.1"])
df_17 = df_17.drop(columns=["month.1"])
df_17 = df_17.drop(columns=["day.1"])
df_17 = df_17.drop(columns=["hour.1"])
df_17 = df_17.drop(columns=["minute.1"])
df_17 = df_17.drop(columns=["second.1"])
df_17 = df_17.drop(columns=["longitude"])
df_17 = df_17.drop(columns=["altitude"])
df_17 = df_17.drop(columns=["elevation"])
df_17 = df_17.drop(columns=["site_gaw_id"])
df_17 = df_17.drop(columns=["latitude"])
df_17 = df_17.drop(columns=["intake_height"])
df_17 = df_17.drop(columns=["flask_no"])
df_17 = df_17.drop(columns=["ORG_QCflag"])
df_17 = df_17.drop(columns=["QCflag"])
df_17 = df_17.drop(columns=["instrument"])
df_17 = df_17.drop(columns=["measurement_method"])
df_17 = df_17.drop(columns=["scale"])
df_17 = df_17.drop(columns=["minute"])
df_17.rename(columns={ "value":"co"},inplace=True)
df_17

In [ ]:
co_df = pd.concat([df_17, df_24]).sort_values('datetime').reset_index(drop=True)

In [ ]:
co_df = co_df.drop(columns=["year"])
co_df = co_df.drop(columns=["month"])
co_df = co_df.drop(columns=["day"])
co_df = co_df.drop(columns=["hour"])
co_df

In [ ]:
# Convert timestamp to datetime
co_df["datetime"] = pd.to_datetime(co_df["datetime"])

# Set the timestamp as the index
co_df.set_index("datetime", inplace=True)

# Resample data hourly and calculate the mean
co_hourly_means = co_df.resample('H').mean()


print(co_hourly_means)

In [ ]:
# Format the index to 'yyyy-mm-dd hh:mm:ss'
co_hourly_means['datetime'] = co_hourly_means.index.strftime('%Y-%m-%d %H:%M:%S')

# Reset index and reformat DataFrame
co_hourly_means.reset_index(drop=True, inplace=True)
co_hourly_means = co_hourly_means[['datetime', 'co']]
#remove lines with NaN
co_hourly_means.dropna(how='any', inplace=True)
co_hourly_means['co_ppt']=co_hourly_means['co']*1000
co_hourly_means

In [ ]:
co_hourly_means['datetime'] = pd.to_datetime(co_hourly_means['datetime'])
prop_df['datetime'] = pd.to_datetime(prop_df['datetime'])
# Merge the datasets on datetime (inner join keeps only matching rows)
co_c3h8 = pd.merge(prop_df, co_hourly_means, on='datetime', how='inner')

#remove lines with NaN
co_c3h8.dropna(how='any', inplace=True)
co_c3h8

In [ ]:
co_c3h8['date'] = pd.to_datetime(co_c3h8['date'])
co_c3h8['MonthDay'] = co_c3h8['date'].dt.strftime('%m-%d')

# Define the three time ranges
time_range_1 = ('03-09', '05-04')
time_range_2 = ('05-05', '10-22')
time_range_3 = ('10-23', '12-29')

# Filter data for each time range
range_1_data = co_c3h8[(co_c3h8['MonthDay'] >= time_range_1[0]) & (co_c3h8['MonthDay'] <= time_range_1[1])]
range_2_data = co_c3h8[(co_c3h8['MonthDay'] >= time_range_2[0]) & (co_c3h8['MonthDay'] <= time_range_2[1])]
range_3_data = co_c3h8[(co_c3h8['MonthDay'] >= time_range_3[0]) & (co_c3h8['MonthDay'] <= time_range_3[1])]

In [ ]:
#To generate boxplots for the three time periods for the year 2020 and compare them with the time range 2011-2019, 2020, 2021-2023
# Extract year, month, and day from 'Date'

# Separate 2020, 2011-2019, and 2021, 2022-2023 data for each time range
def get_year_split(data):
    data_2020 = data[data['year'] == 2020]
    data_2011_2019 = data[(data['year'] >= 2011) & (data['year'] <= 2019)]
    data_2021_2023 = data[(data['year'] >= 2021) & (data['year'] <= 2023)]
    return data_2020,  data_2021_2023, data_2011_2019

range_1_2020,  range_1_2021_2023, range_1_2011_2019  = get_year_split(range_1_data)
range_2_2020, range_2_2021_2023, range_2_2011_2019 = get_year_split(range_2_data)
range_3_2020, range_3_2021_2023,range_3_2011_2019 = get_year_split(range_3_data)


In [ ]:

# Add a 'Period' column to distinguish among 2020, 2021, 2022-2023 and 2011-2019
range_1_2011_2019['Period'] = '2011-2019'
range_1_2020['Period'] = '2020'
range_1_2021_2023['Period'] = '2021-2023'
range_2_2011_2019['Period'] = '2011-2019'
range_2_2020['Period'] = '2020'
range_2_2021_2023['Period'] = '2021-2023'
range_3_2011_2019['Period'] = '2011-2019'
range_3_2020['Period'] = '2020'
range_3_2021_2023['Period'] = '2021-2023'


# Combine the data for each time range
range_1_combined = pd.concat([range_1_2011_2019,range_1_2020,range_1_2021_2023])
range_2_combined = pd.concat([range_2_2011_2019,range_2_2020, range_2_2021_2023])
range_3_combined = pd.concat([range_3_2011_2019, range_3_2020, range_3_2021_2023 ])

In [ ]:
range_1_combined

In [ ]:
#range 1
#  3 time ranges (I Lockdown, mild measures, II lockdown) for the  '2011-2019', '2020', '2021', '2022-2023' years


g=sns.lmplot(x='co_ppt', y='C3H8', data=range_1_combined, robust=True, hue='Period', markers='.',
              scatter_kws={"s": 10}, col='Period', height=4)
#plt.title("Scatter Plot with Linear fit")
titles = {221: '2011-2019', 222: '2020', 223: '2022-2023'}

def annotate(data, **kws):
    r, p = sp.stats.pearsonr(data['co_ppt'], data['C3H8'])
    ax = plt.gca()
    ax.text(.05, .8, 'r={:.2f}, p={:.2g}'.format(r, p),
            transform=ax.transAxes)
    #ax.set_title('Range1')
    ax.set_xlabel('CO [ppt]')
    ax.set_ylabel('$C_3H_8$ [ppt]')
    ax.ticklabel_format(axis='x', style='sci', 
                          scilimits=(0,0))
    
g.map_dataframe(annotate)
plt.subplots_adjust(top=0.85)
plt.suptitle('time_range_1 = from 03-09 to 05-04')

plt.savefig("C:\\YEAR1\\NMVOC\\code\\figures\\c3h8_CO_Covid_year1.png", dpi=300)
plt.show()

In [ ]:
#range 2
#  3 time ranges (I Lockdown, mild measures, II lockdown) for the  '2011-2019', '2020', '2021', '2022-2023' years


g=sns.lmplot(x='co_ppt', y='C3H8', data=range_2_combined, robust=True, hue='Period', markers='.', scatter_kws={"s": 10}, col='Period', height=4)
#plt.title("Scatter Plot with Linear fit")
titles = {221: '2011-2019', 222: '2020', 223: '2022-2023'}

def annotate(data, **kws):
    r, p = sp.stats.pearsonr(data['co_ppt'], data['C3H8'])
    ax = plt.gca()
    ax.text(.05, .8, 'r={:.2f}, p={:.2g}'.format(r, p),
            transform=ax.transAxes)
    #ax.set_title(f'Year: {year}')
    ax.set_xlabel('CO [ppt]')
    ax.set_ylabel('$C_3H_8$ [ppt]')
    ax.ticklabel_format(axis='x', style='sci', 
                          scilimits=(0,0))
    
g.map_dataframe(annotate)

plt.subplots_adjust(top=0.85)
plt.suptitle('time_range_2 = from 05-05 to 10-22')
plt.savefig("C:\\YEAR1\\NMVOC\\code\\figures\\c3h8_CO_Covid_year2.png", dpi=300)
plt.show()
        

In [ ]:
# range 3
# 3 time ranges (I Lockdown, mild measures, II lockdown) for the  '2011-2019', '2020', '2021', '2022-2023' years


g=sns.lmplot(x='co_ppt', y='C3H8', data=range_3_combined, robust=True, hue='Period', markers='.', scatter_kws={"s": 10}, col='Period', height=4)
#plt.title("Scatter Plot with Linear fit")
titles = {221: '2011-2019', 222: '2020', 223: '2022-2023'}

def annotate(data, **kws):
    r, p = sp.stats.pearsonr(data['co_ppt'], data['C3H8'])
    ax = plt.gca()
    ax.text(.05, .8, 'r={:.2f}, p={:.2g}'.format(r, p),
            transform=ax.transAxes)
    #ax.set_title(f'Year: {year}')
    ax.set_xlabel('CO [ppt]')
    ax.set_ylabel('$C_3H_8$ [ppt]')
    ax.ticklabel_format(axis='x', style='sci', 
                          scilimits=(0,0))

g.map_dataframe(annotate)
plt.subplots_adjust(top=0.85)
plt.suptitle('time_range_3 = from 10-23 to 12-29')
plt.savefig("C:\\YEAR1\\NMVOC\\code\\figures\\c3h8_CO_Covid_year3.png", dpi=300)
plt.show()

In [ ]:
#To generate boxplots for the three time periods for the year 2020 and compare them with the time range 2011-2019, 2021-2023
# Extract year, month, and day from 'Date'

# Separate 2020, 2021-2023, 2011-2019 data for each time range
def get_year_split(data):
    data_2020 = data[data['year'] == 2020]
    data_2011_2019 = data[(data['year'] >= 2011) & (data['year'] <= 2019)]
    data_2021_2023 = data[(data['year'] >= 2021) & (data['year'] <= 2023)]
    return data_2020, data_2021_2023, data_2011_2019

range_1_2020, range_1_2021_2023, range_1_2011_2019  = get_year_split(range_1_data)
range_2_2020, range_2_2021_2023, range_2_2011_2019 = get_year_split(range_2_data)
range_3_2020, range_3_2021_2023, range_3_2011_2019 = get_year_split(range_3_data)

In [ ]:
# Add a 'Period' column to distinguish among 2020, 2021, 2022-2023 and 2011-2019
range_1_2011_2019['Period'] = '2011-2019'
range_1_2020['Period'] = '2020'
range_1_2021_2023['Period'] = '2021-2023'
range_2_2011_2019['Period'] = '2011-2019'
range_2_2020['Period'] = '2020'
range_2_2021_2023['Period'] = '2021-2023'
range_3_2011_2019['Period'] = '2011-2019'
range_3_2020['Period'] = '2020'
range_3_2021_2023['Period'] = '2021-2023'


# Combine the data for each time range
range_1_combined = pd.concat([range_1_2011_2019,range_1_2020,range_1_2021_2023])
range_2_combined = pd.concat([range_2_2011_2019,range_2_2020, range_2_2021_2023])
range_3_combined = pd.concat([range_3_2011_2019, range_3_2020,range_3_2021_2023 ])

In [ ]:
#boxplt comparison among 3 time ranges (I Lockdown, mild measures, II lockdown) for the  '2011-2019', '2020', '2021-2023' years

# Define the plotting function
def plot_boxplot_with_mean(ax, data, title):
    sns.boxplot(
        x='Period', y='C3H8', data=data, ax=ax,
        showfliers=False,  # Hide outliers
        whis=[5, 95],      # Whiskers representing 5th–95th percentiles
        showmeans=True, fill=False,
        meanprops={"marker": "D", "markerfacecolor": "red", "markeredgecolor": "black"},
        boxprops={'color': 'black'}, medianprops={'color': 'black'},
        whiskerprops={'color': 'black'}, capprops={'color': 'black'}
    )
    
    # Set the title
    ax.set_title(title, fontsize=14)
    
    # Set x-axis labels
    time_order = ['2011-2019', '2020', '2021-2023']
    ax.set_xticks(range(len(time_order)))
    ax.set_xticklabels(time_order, fontsize=12)
    
    
    ax.set_ylabel('C3H8 [ppt]', fontsize=14)
    ax.set_xlabel('Time', fontsize=14)
    
    # Set y-axis tick size
    ax.tick_params(axis='y', labelsize=12)

    # Set y-axis limits
    ax.set_ylim(0, 1000)

# Calculate and plot the mean values
    means = data.groupby('Period')['C3H8'].mean()
    for i, mean_value in enumerate(means):
        # Add text next to the diamond markers
        ax.text(i, mean_value + 6, f'{mean_value:.1f}', color='black', ha='center', fontsize=12)

# Create the figure
plt.figure(figsize=(14, 8))

# Plot for Time Range 1
ax1 = plt.subplot(1, 3, 1)
plot_boxplot_with_mean(ax1, range_1_combined, 'I lockdown')

# Plot for Time Range 2
ax2 = plt.subplot(1, 3, 2)
plot_boxplot_with_mean(ax2, range_2_combined, 'Mild restrictions')

# Plot for Time Range 3
ax3 = plt.subplot(1, 3, 3)
plot_boxplot_with_mean(ax3, range_3_combined, 'II lockdown')

# Adjust layout to avoid overlapping
plt.tight_layout()

# Show the plot
plt.savefig("C:\\YEAR1\\NMVOC\\code\\figures\\c3h8_CMN_COVID_3_timeRanges.png", dpi=300)
plt.show()


In [ ]:
# Define your time ranges
time_range_1 = ('03-09', '05-04')
time_range_2 = ('05-05', '10-22')
time_range_3 = ('10-23', '12-29')

# Function to filter the data by year and time range
def filter_by_time_range(df, year, time_range):
    start_date = pd.to_datetime(f"{year}-{time_range[0]}")
    end_date = pd.to_datetime(f"{year}-{time_range[1]}")
    return df[(df['date'] >= start_date) & (df['date'] <= end_date)]

# Create lists to hold data for ANOVA
data_range_1, data_range_2, data_range_3 = [], [], []

# Iterate over the time ranges and periods, and collect data
years_groups = [(2011, 2019), (2020, 2020), (2021, 2023)]

for start_year, end_year in years_groups:
    for year in range(start_year, end_year + 1):
        # Collect data for each time period within each year
        data_range_1.append(filter_by_time_range(df, year, time_range_1)['C3H8'].values)
        data_range_2.append(filter_by_time_range(df, year, time_range_2)['C3H8'].values)
        data_range_3.append(filter_by_time_range(df, year, time_range_3)['C3H8'].values)

# Flatten the data (if you have a list of arrays, combine them into one list for each time period)
data_range_1 = [val for sublist in data_range_1 for val in sublist]
data_range_2 = [val for sublist in data_range_2 for val in sublist]
data_range_3 = [val for sublist in data_range_3 for val in sublist]

# Perform one-way ANOVA
f_stat, p_value = stats.f_oneway(data_range_1, data_range_2, data_range_3)

# Print the results
print(f"F-statistic: {f_stat:.4f}")
print(f"P-value: {p_value:.4f}")

# Interpretation of p-value
if p_value < 0.05:
    print("The mean values across the time periods are significantly different.")
else:
    print("No significant difference between the means of the time periods.")

In [ ]:
#To generate boxplots for the three time periods for the year 2020 and compare them with the time range 2011-2019, 2021-2023
# Extract year, month, and day from 'Date'



# Separate 2020, 2021-2023, 2011-2019 data for each time range
def get_year_split(data):
    data_2020 = data[data['year'] == 2020]
    data_2011_2019 = data[(data['year'] >= 2011) & (data['year'] <= 2019)]
    data_2021_2023 = data[(data['year'] >= 2021) & (data['year'] <= 2023)]
    return data_2020, data_2021_2023, data_2011_2019

range_2020, range_2021_2023, range_2011_2019  = get_year_split(df)


In [ ]:
#boxplt comparison among 3 time ranges:  '2011-2019', '2020', '2021-2023' years
# Define time periods: 2011-2019, 2020, 2021-2023
def assign_period(date):
    if pd.Timestamp('2011-01-01') <= date <= pd.Timestamp('2019-12-31'):
        return '2011-2019'
    elif pd.Timestamp('2020-01-01') <= date <= pd.Timestamp('2020-12-31'):
        return '2020'
    elif pd.Timestamp('2021-01-01') <= date <= pd.Timestamp('2023-12-31'):
        return '2021-2023'
    else:
        return None

# Apply this to your dataframe to create a 'Period' column
df['Period'] = pd.to_datetime(df['date']).apply(assign_period)

# Function to plot the boxplot and display the mean values
def plot_boxplot_with_mean(data, ax):
    sns.boxplot(
        x='Period', y='C3H8', data=data, ax=ax,
        showfliers=False,  # Hide outliers
        whis=[5, 95],      # Whiskers representing 5th–95th percentiles
        showmeans=True,
        fill=False,
        meanprops={"marker": "D", "markerfacecolor": "red", "markeredgecolor": "black"},
        boxprops={'color': 'black'}, medianprops={'color': 'black'},
        whiskerprops={'color': 'black'}, capprops={'color': 'black'}
    )

    # Calculate the mean values for each period
    means = data.groupby('Period')['C3H8'].mean()

    # Add the mean values above the diamond markers
    for i, (period, mean_value) in enumerate(means.items()):
        ax.text(i, mean_value + 5, f'{mean_value:.2f}', color='black', ha='center', fontsize=10)

    # Set labels and limits
    ax.set_ylabel('$C_3H_8$ [ppt]', fontsize=14)
    ax.set_xlabel('Time', fontsize=14)
    ax.set_ylim(0,  1000)  # Adjust y-axis limit based on the data range
    ax.tick_params(axis='y', labelsize=12)
    #ax.set_title('C3H8 Levels by Time Range', fontsize=16)

# Create the figure for the boxplot
plt.figure(figsize=(8, 6))

# Create a single subplot for the three periods
ax = plt.subplot(1, 1, 1)
plot_boxplot_with_mean(df, ax)

# Adjust layout
plt.tight_layout()


# Show the plot
plt.savefig("C:\\YEAR1\\NMVOC\\code\\figures\\c3h8_CMN_COVID_yearRanges.png", dpi=300)
#plt.show()


In [ ]:
median = df.groupby('Period')['C3H8'].quantile(0.5)
median